In [0]:
import nltk
from nltk import *
nltk.download('wordnet')

In [0]:
!pip3 install gensim
!pip install pulp

In [0]:
#kamal
import numpy as np
import pandas as pd
import math
import copy
import sys
import sklearn.metrics as skm

#nalla
from itertools import product
from collections import defaultdict
import csv
import nltk
from nltk.corpus import stopwords
import gensim
# from pyemd import emd
from gensim import corpora, models, similarities
from scipy.spatial.distance import euclidean
import pulp

#sowrya
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from six import iteritems
from collections import deque

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Paraphrase Detection on Twitter

## Reading data
* each tweet is considered as a document
* making a list of all the tweets and finding the idf values

In [0]:
def glove_processing():
    f = open('./drive/My Drive/glove/glove25.txt','r')
    model = {}
    for line in f:
        line = line.split()
        word = line[0]
        embedding = np.array([float(val) for val in line[1:]])
        model[word] =  embedding
    print(len(model))
    return model

In [0]:
df = pd.read_csv('./drive/My Drive/Data/data/train.data', sep='\t', lineterminator='\n', names=['Topic_Id', 'Topic_Name',  'Sent_1', 'Sent_2', 'Label', 'Sent_1_tag','Sent_2_tag'])
df2 = df[['Sent_1', 'Sent_2', 'Label']]
model = glove_processing()

In [0]:
print(df2)

## Preprocessing Tweets

* Preprocess the tweets with respect to .... to get a 4D vector

In [0]:
def process_labels(labels):
    '''returns binary labels 0 or 1'''
    new_labels = np.array([])
    for i in labels:
        if type(i) == str and len(i) > 1:
            i = i[1]
        if int(i) >= 3:
            new_labels = np.append(new_labels, int(1))
        else:
            new_labels = np.append(new_labels, int(0))
    return new_labels

In [0]:
train_labels = process_labels(df2.Label)
print(train_labels.shape)

### Cosine Distance for average of sentence vectors

In [0]:
def sent_vec(sent, model):
    ans = np.zeros(25)
    count = 0
    if len(sent) == 0:
        return ans
    for i, word in enumerate(sent.split()):
        if word not in model:
            continue
        else:
            ans += np.array(model[word])
            count += 1
    if count > 0:
        ans = ans / count
    return ans, count


def cosine_dist(vec1, vec2):
    ans = 0
    mod1 = 0
    mod2 = 0
    for i in range(vec1.size):
        ans += vec1[i] * vec2[i]
        mod1 += vec1[i] * vec1[i]
        mod2 += vec2[i] * vec2[i]
    if ans == 0:
        return 0
    return ans / (math.sqrt(mod1) * math.sqrt(mod2))


def cosine_data(df, model):
    arr = np.array([]) #has the cosine distance values for all the tweet pairs
    for i in range(df.Sent_1.size):
        vec1, count1 = sent_vec(df.Sent_1[i], model)
        vec2, count2 = sent_vec(df.Sent_2[i], model)
        ans = cosine_dist(vec1, vec2)
        arr = np.append(arr, ans)
#         if i == 7:
#             print(i, df.Sent_1[i], "sdfsdf", df.Sent_2[i], vec1, vec2, count1, count2, ans)
    return arr

### Word movers distance

In [0]:
import nltk
nltk.download('stopwords')
singleindexing = lambda m, i, j: m*i+j
unpackindexing = lambda m, k: (k/m, k % m)

def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}

# use PuLP
def word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs = {}
    for token in all_tokens:
        try:
            wordvecs[token] = wvmodel[token]
        except KeyError:
            wordvecs[token] = 0

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T= pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    try:
        prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    except KeyError:
        prob+=0
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)
    prob.solve()
    if(prob == None):
        print("yes")
    return prob

def word_mover_distance(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

def arr2_creator(df, model):
    arr2 = np.array([])
    stop_words = stopwords.words('english')
    dist=0
    l = open('./drive/My Drive/Data/data/train.data','r')
    j = 0
    total = 0
    for i in range(df.Sent_1.size):
        sent1 = df.Sent_1[i].split()
        sent2 = df.Sent_2[i].split()
        sent1 = [w for w in sent1 if w not in stop_words]
        sent2 = [w for w in sent2 if w not in stop_words]
        temp = word_mover_distance(sent1, sent2, model)
        if temp == None:
            temp = total / j
        else:
            total += temp
            j += 1
        val = 0
        if temp == 0:
            val = 2
        else:
            val = 1/temp
        arr2 = np.append(arr2, val)
    return arr2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Wordnet based noun similarity

In [0]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
def penn_to_wn(tag):
    if tag.startswith('N'):
        return 'n'
    # if tag.startswith('V'):
    #     return 'v'
    # if tag.startswith('J'):
    #     return 'a'
    # if tag.startswith('R'):
    #     return 'r'
    return None

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
#         print('here')
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
        print('here')
    except:
#         print(wn.synsets(word, wn_tag))
#         print('here_1')
        return None

def shortest_hypernym_paths(synset):
    # hyp = synset.hypernyms()
    # hyp_1 = synset.instance_hypernyms()
    if synset._name == '*ROOT*':
        return {synset: 0}
    queue = deque([(synset,0)])
    path = {}
    while queue:
        s, depth = queue.popleft()
        if s in path:
            continue
        path[s] = depth
        depth += 1
        queue.extend((hyp, depth) for hyp in s.hypernyms())
        queue.extend((hyp, depth) for hyp in s.instance_hypernyms())
    # simulate_root = True
    # # if simulate_root:
    # #     fake_synset = wn.synset(None)
    # #     fake_synset._name = '*ROOT*'
    # #     path[fake_synset] = max(path.values()) + 1
    return path

def shortest_path_distance(synset_1,synset_2):
    if synset_1 == synset_2:
        return 0
    dist_dict1 = shortest_hypernym_paths(synset_1)
    dist_dict2 = shortest_hypernym_paths(synset_2)
    inf = float('inf')
    path_distance = inf
    for synset, d1 in iteritems(dist_dict1):
        d2 = dist_dict2.get(synset, inf)
        path_distance = min(path_distance, d1 + d2)
    return None if math.isinf(path_distance) else path_distance

def path_similarity(synset_1,synset_2):
    distance = shortest_path_distance(synset_1,synset_2)
    if distance is None or distance < 0:
        return None
    return 1.0/(distance + 1)

def sentence_similarity(sentence1, sentence2):
    sentence1, sentence2 = pos_tag(word_tokenize(sentence1)), pos_tag(word_tokenize(sentence2))
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    for syn1 in synsets1:
        arr_simi_score = []
        for syn2 in synsets2:
            simi_score = path_similarity(syn1,syn2)
            if simi_score is not None:
                arr_simi_score.append(simi_score)
        if(len(arr_simi_score) > 0):
            best = max(arr_simi_score)
            score += best
            count += 1
    if count == 0:
        score = 0
    else:
        score /= count
    return score

def arr3_creator(df):
    arr3 = np.array([])
    for i in range(df.Sent_1.size):
        val = (sentence_similarity(df.Sent_1[i], df.Sent_2[i]) + sentence_similarity(df.Sent_2[i], df.Sent_1[i]))/2
        arr3 = np.append(arr3, val)
    return arr3
  
  
# def arr3_creator(df):
#     arr3 = np.array([])
#     s_1 = "EJ Manuel the 1st QB to go in this draft"
#     s_2 = "if EJ is the 1st QB off the board"
#     for i in range(df.Sent_1.size):
# #     val = (sentence_similarity(s_2, s_1) + sentence_similarity(s_1, s_2))/2
#         val = (sentence_similarity(df.Sent_1[i], df.Sent_2[i]) + sentence_similarity(df.Sent_2[i], df.Sent_1[i]))/2
# #         print(df.Sent_1[i], df.Sent_2[i], val)
#         arr3 = np.append(arr3, val)
#     return arr3

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Creating vectors
* Appending all values for each tweet pair to create a 4D vector 

In [0]:
arr1 = cosine_data(df2, model)
print(arr1.shape)

(13063,)


In [0]:
arr2 = arr2_creator(df2, model)
print(arr2.shape)

(13063,)


In [0]:
arr3 = arr3_creator(df2)
print(arr3.shape)

(13063,)


In [0]:
print(arr1[:100])
print(arr2[:100])
print(arr3[:100])

In [0]:
def create_vectors(arr1, arr2, arr3):
    new_arr = np.vstack((arr1, arr2, arr3))
    return new_arr

In [0]:
train_data = create_vectors(arr1, arr2, arr3) #(3,N) dimensions
print(train_data.shape)
#each column is a vector

(3, 13063)


## Training Models

### Logistic Regression

In [0]:
def loss(model, train_data, train_labels):
    ans = 0
    for i in range(train_labels.size):
#         print(np.dot(model, train_data[:, i]))
        val = sigmoid(np.dot(model, train_data[:, i]))
        ans += (train_labels[i] * np.log(val)) + ((1-train_labels[i]) * np.log(1-val))
    return abs(ans)

def classify(val):
    if val >= 0.5:
        return 1
    else:
        return 0

def sigmoid(val):
    ans = 1 / (1 + np.exp(-val))
    return ans

def loss_gradient(model, train_data, train_labels):
    loss = np.zeros(model.size)
    for j in range(train_labels.size):
#         print(j, train_data[:, j])
        loss = loss + train_data[:, j] * (sigmoid(np.dot(model, train_data[:, j])) - train_labels[j])
#         if loss[1] == None:
#             print (j, train_data[:,j])
    return loss

def log_reg(train_data, train_labels):
    '''returns a trained model'''
    eta = 0.00005
    model = np.random.rand(3)
    curr_loss = loss(model, train_data, train_labels)
    prev_loss = curr_loss - 100
    while abs(curr_loss - prev_loss) > 1:
        print(abs(curr_loss - prev_loss))
#     while loss(model, train_data, train_labels) > 0.1:
        val = loss_gradient(model, train_data, train_labels)
        model -= eta * val
        prev_loss = curr_loss
        curr_loss = loss(model, train_data, train_labels)
    return model

In [0]:
lr_model = log_reg(train_data, train_labels)
# print(lr_model)

In [0]:
print(lr_model)

[-1.42810655  0.67765735  0.29323318]


# testing

In [0]:
df = pd.read_csv('./drive/My Drive/Data/data/test.data', sep='\t', lineterminator='\n', names=['Topic_Id', 'Topic_Name',  'Sent_1', 'Sent_2', 'Label', 'Sent_1_tag','Sent_2_tag'])
test_pairs = df[['Sent_1', 'Sent_2']]
# print(np.shape(test_pairs))
test_labels = process_labels(df.Label)
print(df.Sent_1)

In [0]:
test_arr1 = cosine_data(test_pairs, model)
print(test_arr1.shape)

(972,)


In [0]:
test_arr2 = arr2_creator(test_pairs, model)
print(test_arr2.shape)

(972,)


In [0]:
test_arr3 = arr3_creator(test_pairs)
print(test_arr3.shape)

(972,)


In [0]:
test_data = create_vectors(test_arr1, test_arr2, test_arr3)

In [0]:
def testing(model, test_data, test_labels):
    predict = np.array([])
    for i in range(test_labels.size):
        val = classify(sigmoid(np.dot(model, test_data[:, i])))
        predict = np.append(predict, val)
    return predict

In [0]:
predicted_arr = testing(lr_model, test_data, test_labels)
print(skm.classification_report(test_labels, predicted_arr))

In [0]:
print(skm.accuracy_score(test_labels, predicted_arr))
# print(train_data[2])

## MLP


In [0]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

In [0]:
def MLP(train_data, train_labels, test_data, test_labels):
    X_train = train_data.T
    y_train = train_labels
    y_binary = to_categorical(y_train)

    scale = np.max(X_train)
    X_train /= scale

    mean = np.std(X_train)
    X_train -= mean

    input_dim = X_train.shape[1]
#     print?(X_train[0])
#     print(input_dim)
    nb_classes = 1
    
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(nb_classes))
    model.add(Activation('sigmoid'))
#     model = Sequential()
#     model.add(Dense(64, input_dim=input_dim, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(1, activation='sigmoid'))

    # we'll use categorical xent for the loss, and RMSprop as the optimizer
    model.compile(loss='mse', optimizer='rmsprop')

    print("Training...")
    model.fit(X_train, y_train, nb_epoch=50, batch_size=128, verbose=2)
    
    X_test = test_data.T
    y_test = test_labels
    preds = model.predict_classes(X_test, verbose=0)

    score = model.evaluate(X_test, y_test, batch_size=128)
    print("score",score)
    return preds, model
# print("Generating test predictions...")
# preds = model.predict_classes(X_test, verbose=0)

In [0]:
preds = np.array([])
preds, mlp_model = MLP(train_data, train_labels, test_data, test_labels)
# print(mlp_model)

In [0]:
# predicted_arr = testing(mlp_model, test_data, test_labels)
print(skm.classification_report(test_labels, preds))
print(skm.accuracy_score(test_labels, preds))

In [0]:
from sklearn.externals import joblib
joblib.dump(model, "./drive/My Drive/Data/" + (str)model+ ".joblib")

In [0]:
print(df2.Sent_1)

**RNN**

In [0]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, LSTM, Lambda, Dense,concatenate
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization

In [0]:
TRAIN_CSV = './drive/My Drive/Data/data/train.data'
TEST_CSV = './drive/My Drive/Data/data/test.data'
EMBEDDING_FILE = './drive/My Drive/glove/glove25.txt'
MODEL_SAVING_DIR = './drive/My Drive/Data/'

In [0]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def prep_embs(train_df, test_df, word2vec):
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
    # word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

    questions_cols = ['Sent_1', 'Sent_2']

    # Iterate over the questions only of both training and test datasets
    for dataset in [train_df, test_df]:
        for index, row in dataset.iterrows():
            for question in questions_cols:
                q2n = []  # q2n -> question numbers representation
                for word in text_to_word_list(row[question]):

                    # Check for unwanted words
                    if word in stops and word not in word2vec:
                        continue

                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        q2n.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        q2n.append(vocabulary[word])

                # Replace questions as word to question as number representation
                dataset.set_value(index, question, q2n)

    embedding_dim = 25
    embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabulary.items():
        if word in word2vec:
            embeddings[index] = word2vec[word]
    return embeddings, embedding_dim


In [0]:
embeddings, embedding_dim = prep_embs(df2, df, model)
print(len(embeddings))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


9515


In [0]:
max_seq_length = max(df2.Sent_1.map(lambda x: len(x)).max(),
                     df2.Sent_2.map(lambda x: len(x)).max(),
                     test_pairs.Sent_1.map(lambda x: len(x)).max(),
                     test_pairs.Sent_2.map(lambda x: len(x)).max())

# print(max_seq_length)
# Split to train validation
validation_size = 0.1
# training_size = len(df2) - validation_size

questions_cols = ['Sent_1', 'Sent_2']

X = df2[questions_cols]
Y = train_labels

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)
X1 = X_train
# print(np.shape(Y_train))
# Split to dicts
X_train = {'left': X1.Sent_1, 'right': X1.Sent_2}
X_validation = {'left': X_validation.Sent_1, 'right': X_validation.Sent_2}
X_test = {'left': df.Sent_1, 'right': df.Sent_2}
# print(np.shape(X_train['left']))
# print(df.Sent_1)
i = 0
for dataset, side in itertools.product([X_train, X_validation, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)
#     print(np.size(dataset[side]))
#     print(np.shape(X_train['left']))


# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [0]:
print(np.shape(X_train['left']))
print(np.shape(X_test['left']))

(11756, 101)
(972, 101)


In [0]:
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 50
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')
# l = Sequential()
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
merged_vector = concatenate([left_output, right_output], axis=-1)
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# malstm_distance = BatchNormalization()(malstm_distance)
out = Dense(1, activation='sigmoid')(malstm_distance)
# # Pack it all up into a model
malstm = Model([left_input, right_input],outputs=out)
# malstm.add(Dense(1, activation='sigmoid'))
# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

# malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
malstm.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [0]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [0]:
# print(X_test['input_1'])
# X_test = np.reshape(X_test, (np.shape(X_test['input_1']), 1, np.shape(X_test['input_2'])))
preds = np.array([])
preds = malstm.predict([X_test['left'], X_test['right']])
j = 0
correct = 0
incorrect = 0
preds2 = []
for i in preds:
  ret = classify(i)
  preds2.append(ret)
  if ret == test_labels[j]:
    correct += 1
  else:
    incorrect += 1
  j+=1
#   print(classify(i), test_labels[j])
print(correct/ (correct + incorrect))

0.6882716049382716


In [0]:
malstm.evaluate([X_test['left'], X_test['right']], test_labels)

972/972 [==============================] - 1s 715us/step


[0.6188942833569805, 0.6882716049382716]

In [0]:
print(skm.classification_report(test_labels, preds2))

             precision    recall  f1-score   support

        0.0       0.69      0.97      0.81       663
        1.0       0.57      0.07      0.13       309

avg / total       0.66      0.69      0.59       972

